<h1 style="text-align: center;"> Учебно-исследовательская работа по теме <br>"Разработка решения для сегментации изображений <br> на экран и окружающую область"</h1>

<h2><b>Подготовка датасета</b></h2>

В этом разделе мы подготовим наш датасет для обучения модели в формате <b>TFRecord</b> для <b>TensorFlow</b>.

Для поиска датасета воспользуемся ресурсом датасетов "<a href="https://universe.roboflow.com">Roboflow</a>"

Установим библиотеку для работы с Roboflow:


In [3]:
!pip install roboflow


Теперь используем Roboflow для загрузки нужного датасета в формате TFRecord (в директории должна создаться папка "Computer-monitor-3"):

In [4]:
from roboflow import Roboflow

# Ваш API ключ Roboflow
rf = Roboflow(api_key="to3YrOBj0FlHoBjF9Vwn")

# Указываем workspace и проект
project = rf.workspace("object-detection-bwxix").project("computer-monitor")

# Загружаем нужную версию датасета
version = project.version(3)  # Выбираем версию датасета
dataset = version.download("tfrecord")  # Скачиваем данные в формате TFRecord


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Computer-monitor-3 in tfrecord:: 100%|██████████| 11/11 [00:00<00:00, 140.65it/s]


Теперь установим TensorFlow:

In [5]:
!pip install tensorflow

Используем загруженные данные для TensorFlow:

In [ ]:
import tensorflow as tf

# Путь к файлу TFRecord, который был загружен
tfrecord_file = dataset.location + "/dataset.tfrecord"

# Чтение и обработка данных из TFRecord
def _parse_function(proto):
    keys_to_features = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'mask': tf.io.FixedLenFeature([], tf.string),
    }
    
    parsed_features = tf.io.parse_single_example(proto, keys_to_features)
    
    # Декодируем изображение и маску
    image = tf.image.decode_jpeg(parsed_features['image'])
    mask = tf.image.decode_png(parsed_features['mask'], channels=1)
    
    return image, mask

# Загружаем данные
dataset = tf.data.TFRecordDataset(tfrecord_file)
dataset = dataset.map(_parse_function)

# Применяем преобразования: изменение размера, нормализация и пакетирование
dataset = dataset.map(lambda image, mask: (tf.image.resize(image, [256, 256]), tf.image.resize(mask, [256, 256])))
dataset = dataset.map(lambda image, mask: ((image / 255.0), mask))

# Пакетируем данные
dataset = dataset.batch(32).shuffle(buffer_size=1000)

# Пример вывода пакета данных
for image, mask in dataset.take(1):
    print("Пакет изображений:", image.shape)
    print("Пакет масок:", mask.shape)
